Based on: https://arxiv.org/abs/1508.06576

In [1]:
!ln -s "/content/drive/MyDrive/meu/imgs" ./data

In [2]:
# from __future__ import print_function

import torch
import torch.nn as nn
import torch.optim as optim
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image

# import torch.nn.functional as F
# import matplotlib.pyplot as plt
# import copy
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_size =  356
loader = transforms.Compose(
    [transforms.Resize((img_size,img_size)),
     transforms.ToTensor(),
     #transforms.Normalize(mean=[],std=[])
     ]
)

In [ ]:
# cnn = models.vgg19(pretrained=True).to(device).eval()# get all NET
# cnn = models.vgg19(pretrained=True).features.to(device).eval()# get only the CNN

In [ ]:
"""
import torch.nn.functional as F
def read_img(path,device):
    img = cv2.cvtColor(cv2.imread(path),cv2.COLOR_BGR2RGB) # inportando imagem
    img = torch.Tensor(img) # convertendo em tensor
    img = img.permute(2,0,1).unsqueeze(0) # reordenando
    img = F.interpolate(img, size=224) # resize
    # img = img.to('cpu')
    return img.to(device)
"""

In [4]:
def load_image(path,device,loader):
    img = Image.open(path)
    img = loader(img).unsqueeze(0)
    return img.to(device)

In [5]:
class VGG(nn.Module):
    def __init__(self,device):
        super(VGG, self).__init__()
        self.features_layers = [0,5,10,19,28]
        self.model = models.vgg19(pretrained=True).features[:29].to(device).eval()
    def forward(self, x):
        features = []
        for num, layer in enumerate(self.model):
            x=layer(x)

            if num in self.features_layers :
                features.append(x)
        return features
        

### Import images

In [23]:
original_img = load_image('/content/data/foto.jpeg',device=device,loader=loader)
style_img = load_image('/content/data/AP_Biblioteca_R01-960x540.jpg',device=device,loader=loader)
generated_img = original_img.clone().requires_grad_(True)
model = VGG(device)

### Hyperparametrs

In [26]:
steps = 600
learn_rate = 0.01
alpha = 0.56 # content_weight
beta = 1000000 #0.56 # style_weight
optimizer = optim.Adam([generated_img],lr=learn_rate)

In [27]:
for step in range(steps):
    generated_img_features = model(generated_img)
    original_img_features = model(original_img)
    style_img_features = model(style_img)

    style_loss = original_loss = 0

    for gen_feature, orig_feature, style_feature in zip(
        generated_img_features, original_img_features, style_img_features):

        bash_size, channel, h,w =  gen_feature.shape

        original_loss += torch.mean((gen_feature-orig_feature)**2) 

        #compute gram matrix
        G = gen_feature.view(channel,h*w).mm(
            gen_feature.view(channel,h*w).t())
        
        A = style_feature.view(channel,h*w).mm(
            style_feature.view(channel,h*w).t())

        style_loss += torch.mean((G-A)**2) 

    total_loss = alpha * original_loss + beta * style_loss
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    if step % 50 == 0:
        print(f"{step}: {total_loss.item()}")
        save_image(generated_img,f"hoovertowernight_{step}.png")



0: 47414312435712.0
50: 5152345948160.0
100: 3938927247360.0
150: 3579130413056.0
200: 3358146691072.0
250: 3186107875328.0
300: 3035753349120.0
350: 2894838628352.0
400: 2755863511040.0
450: 2603556536320.0
500: 2434341011456.0
550: 2243671621632.0
